# Preprocess Dataset

In [1]:
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List
import re
import numpy as np
import tempfile

/var/folders/vm/77wrgjgj5wzbyghx353b7gym0000gn/T/ipykernel_15134/3422228704.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

## Extract JD Attributes

In [4]:
SYSTEM_MESSAGE = '''
You are an expert information extraction AI. Your task is to analyze the provided job descriptions and extract specific attributes related to job requirements and compensation.

Please extract the following attributes and format the output *only* as a list of JSON objects with the following properties:

1.  `experience_requirements`: A single string summarizing the required or preferred years and types of experience. Identify and concatenate the relevant phrases or sentences directly from the text. Keep the summary concise but primarily use the original wording. Ensure proper spacing and sentence structure between concatenated parts for readability, making only minimal adjustments (like adding spaces) as needed. If no specific experience requirements are mentioned, use `""` (an empty string).
2.  `skill_requirements`: A single string summarizing the key technical skills and important soft skills mentioned as required or preferred. Identify and concatenate the relevant phrases or sentences directly from the text. Keep the summary concise but primarily use the original wording. Ensure proper spacing and sentence structure between concatenated parts for readability, making only minimal adjustments (like adding spaces) as needed. If no specific skills are mentioned, use `""` (an empty string).
3.  `education_requirements`: A single string summarizing the required or preferred educational qualifications. Identify and concatenate the relevant phrases or sentences directly from the text. Keep the summary concise but primarily use the original wording. Ensure proper spacing and sentence structure between concatenated parts for readability, making only minimal adjustments (like adding spaces) as needed. If no specific education requirements are mentioned, use `""` (an empty string).
4.  `certification_requirements`: A single string summarizing any specific certifications required or preferred. Identify and concatenate the relevant phrases or sentences directly from the text. Keep the summary concise but primarily use the original wording. Ensure proper spacing and sentence structure between concatenated parts for readability, making only minimal adjustments (like adding spaces) as needed. If no certifications are mentioned, use `""` (an empty string).
5.  `min_salary`: Extract the minimum salary figure **only if explicitly stated** in the text. Look for explicit ranges (e.g., "$100k - $120k", "$29.81 - $30.03 an hour", "90,000 to 110,000 annually") or explicit minimums ("starts at 90,000"). Identify the **lower numerical value** from the range or the stated minimum. Remove any currency symbols ($, €, etc.), commas, and surrounding text. Convert 'k' notation to thousands (e.g., "100k" becomes "100000"). Output this cleaned numerical value **as a string** (e.g., "100000", "29.81", "90000"). **Do NOT infer, predict, estimate, or calculate** this value. If no numerical minimum salary or explicit lower bound of a range is stated, this field **must** be `""` (an empty string).
6.  `max_salary`: Extract the maximum salary figure **only if explicitly stated** in the text. Look for explicit ranges (e.g., "$100k - $120k", "$29.81 - $30.03 an hour", "90,000 to 110,000 annually") or explicit maximums ("up to 130,000"). Identify the **upper numerical value** from the range or the stated maximum. Remove any currency symbols ($, €, etc.), commas, and surrounding text. Convert 'k' notation to thousands (e.g., "120k" becomes "120000"). Output this cleaned numerical value **as a string** (e.g., "120000", "30.03", "130000"). **Do NOT infer, predict, estimate, or calculate** this value. If no numerical maximum salary or explicit upper bound of a range is stated, this field **must** be `""` (an empty string).
7.  `salary`: Extract a single representative salary figure **only if explicitly stated** as a specific amount (e.g., "$110,000 per year", "$55.50/hour", "100k annually"). **Do NOT extract ranges into this field.** Identify the numerical value, remove any currency symbols ($, €, etc.), commas, and surrounding text (like 'per year', '/hour', 'annually'). Convert 'k' notation to thousands (e.g., "110k" becomes "110000"). Output this cleaned numerical value **as a string** (e.g., "110000", "55.50"). **Do NOT infer, predict, estimate, or calculate** this value. If no single numerical specific salary figure is explicitly stated, this field **must** be `""` (an empty string).
8.  `pay_period`: Extract the time period **only if explicitly stated** alongside a salary figure in the text (e.g., "per year", "annually", "per hour", "hourly", "per month", "per week", "bi-weekly"). The value must be one of ["YEARLY", "HOURLY", "MONTHLY", "WEEKLY", "BIWEEKLY"]. Map equivalent terms (e.g., "per year" maps to "YEARLY"). **Do NOT infer** the pay period based on the salary magnitude or any other factor. If no period is explicitly stated with an extracted salary figure, or if no salary is mentioned at all, use `""` (an empty string).
9.  `job_id`: Extract only the numeric ID value from the job description XML tags. The input will contain job descriptions wrapped in tags like `<job id=\"123456\">...</job>`. Extract only the ID value (e.g., \"123456\") without any XML tags or quotes. This is a required field and should not be empty.
'''

In [5]:
class ParsedJobDescription(BaseModel):
    job_id: str
    experience_requirements: str
    skill_requirements: str
    education_requirements: str
    certification_requirements: str
    min_salary: str
    max_salary: str
    salary: str
    pay_period: str

class ParsedJobDescriptions(BaseModel):
    jobs: List[ParsedJobDescription]


def extract_jd_attributes(job_ids: list[str], job_descriptions: list[str]) -> ParsedJobDescriptions | None:
    resp = client.beta.chat.completions.parse(
        model="gemini-2.0-flash-lite",
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {
                "role": "user",
                "content": "\n".join(
                    [f"<job id=\"{id}\">{desc}</job>" for id, desc in zip(job_ids, job_descriptions)]
                )
            }
        ],
        temperature=0.5,
        response_format=ParsedJobDescriptions,
    )
    return resp.choices[0].message.parsed

def save_dataframe_safely(df: pd.DataFrame, filepath: str):
    """
    Save a DataFrame to CSV in a transactional manner so that if there is an interruption during the save, 
        all rows will still be saved.
    """
    with tempfile.NamedTemporaryFile(mode='w', suffix='.csv') as tmp_file:
        temp_path = tmp_file.name
        try:
            # Save to the temporary file
            df.to_csv(temp_path, index=False)
            
            # This is guaranteed to be atomic
            os.replace(temp_path, filepath)
        except Exception as e:
            raise e
        
def parse_float(value: str) -> float:
    """Convert a string to a float"""
    return float(value.replace(',', ''))

def clean_jd(text) -> str:
    """Clean a JD text, removing control characters"""
    if not isinstance(text, str): return ""
    return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]', '', str(text))


In [6]:
EXTRACTED_EXPERIENCE_REQUIREMENT_COL = 'extracted_experience_requirement'
EXTRACTED_SKILL_REQUIREMENT_COL = 'extracted_skill_requirement'
EXTRACTED_EDUCATION_REQUIREMENT_COL = 'extracted_education_requirement'
EXTRACTED_CERTIFICATION_REQUIREMENT_COL = 'extracted_certification_requirement'
EXTRACTED_MIN_SALARY_COL = 'extracted_min_salary'
EXTRACTED_MAX_SALARY_COL = 'extracted_max_salary'
EXTRACTED_SALARY_COL = 'extracted_salary'
EXTRACTED_PAY_PERIOD_COL = 'extracted_pay_period'
EXTRACTED_DATETIME_COL = 'extracted_datetime'
EXTRACTED_COLS = [
    EXTRACTED_EXPERIENCE_REQUIREMENT_COL,
    EXTRACTED_SKILL_REQUIREMENT_COL,
    EXTRACTED_EDUCATION_REQUIREMENT_COL,
    EXTRACTED_CERTIFICATION_REQUIREMENT_COL,
    EXTRACTED_MIN_SALARY_COL,
    EXTRACTED_MAX_SALARY_COL,
    EXTRACTED_SALARY_COL,
    EXTRACTED_PAY_PERIOD_COL,
    EXTRACTED_DATETIME_COL
]

In [7]:
df_jobs_clean = pd.read_csv("data/jobs_clean.csv")
df_jobs_clean

,job_id,title,location,location_state,description,formatted_work_type,formatted_experience_level,remote_allowed,company_industries,company_country,company_state,company_city,company_description,company_employee_count,normalized_salary,min_salary,max_salary,med_salary,pay_period
0,921716,Marketing Coordinator,"Princeton, NJ",NJ,Job descriptionA leading real estate firm in N...,Full-time,Unknown,0.0,Real Estate,US,NJ,Jersey City,With years of experience helping local buyers ...,402.0,38480.0,17.0,20.0,NaN,HOURLY
1,1829192,Mental Health Therapist/Counselor,"Fort Collins, CO",CO,"At Aspen Therapy and Wellness , we are committ...",Full-time,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,NaN,NaN,83200.0,30.0,50.0,NaN,HOURLY
2,10998357,Assitant Restaurant Manager,"Cincinnati, OH",OH,The National Exemplar is accepting application...,Full-time,Unknown,0.0,Restaurants,US,Ohio,Mariemont,"In April of 1983, The National Exemplar began ...",15.0,55000.0,45000.0,65000.0,NaN,YEARLY
3,23221523,Senior Elder Law / Trusts and Estates Associat...,"New Hyde Park, NY",NY,Senior Associate Attorney - Elder Law / Trusts...,Full-time,Unknown,0.0,Law Practice,US,New York,Lake Success,"Abrams Fensterman, LLP is a full-service law f...",222.0,157500.0,140000.0,175000.0,NaN,YEARLY
4,35982263,Service Technician,"Burlington, IA",IA,Looking for HVAC service tech with experience ...,Full-time,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,NaN,NaN,70000.0,60000.0,80000.0,NaN,YEARLY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119217,3906267117,Title IX/Investigations Attorney,"Walnut Creek, CA",CA,Our Walnut Creek office is currently seeking a...,Full-time,Mid-Senior level,0.0,Law Practice,US,California,Fresno,"For more than 30 years, Lozano Smith has serve...",185.0,157500.0,120000.0,195000.0,NaN,YEARLY
119218,3906267126,"Staff Software Engineer, ML Serving Platform",United States,Unknown,About Pinterest:\n\nMillions of people across ...,Full-time,Mid-Senior level,1.0,Software Development,US,California,San Francisco,Pinterest's mission is to bring everyone the i...,8667.0,NaN,NaN,NaN,NaN,NaN
119219,3906267131,"Account Executive, Oregon/Washington","Spokane, WA",WA,Company Overview\n\nEPS Learning is a leading ...,Full-time,Mid-Senior level,1.0,Education Administration Programs,US,MD,Bethesda,EPS Learning has partnered with educators for ...,127.0,NaN,NaN,NaN,NaN,NaN
119220,3906267195,Business Development Manager,"Texas, United States",Unknown,The Business Development Manager is a 'hunter'...,Full-time,Unknown,1.0,Industrial Machinery Manufacturing,GB,Lancashire,Rochdale,Trelleborg Applied Technologies manufactures a...,47.0,NaN,NaN,NaN,NaN,NaN


In [8]:
df_extracted_job_attributes = pd.read_csv('data/extracted_job_attributes.csv').drop_duplicates(subset=['job_id'], keep='last')
df_extracted_job_attributes

,job_id,extracted_experience_requirement,extracted_skill_requirement,extracted_education_requirement,extracted_certification_requirement,extracted_min_salary,extracted_max_salary,extracted_salary,extracted_pay_period,extracted_datetime
0,23221523,10-15 years of experienceExperience with vario...,Strong analytical and problem-solving skillsAb...,Juris Doctor degree (J.D.) from an accredited ...,NaN,140000.0,175000.0,NaN,NaN,2025-04-16 23:12:49
1,35982263,Minimum 5 yrs. on the job with mechanical lice...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-15 20:13:33
2,91700727,NaN,"Strong interest in economic development, city ...",Currently enrolled in a graduate or undergradu...,NaN,14.0,20.0,NaN,NaN,2025-04-16 00:43:02
3,103254301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-16 00:44:56
4,112576855,Professional training certification or a minim...,"Excellent communication skills are required, i...","High School degree required, though we will co...",CFC and other trade certifications are desired.,NaN,NaN,NaN,NaN,2025-04-16 00:45:22
...,...,...,...,...,...,...,...,...,...,...
119229,3906258042,Two years of auto appraisal experience preferr...,Strong mathematical and negotiation skills.,"Associates Degree, Bachelors Degree or Technic...",State appraiser and claim handling licenses (w...,61600.0,101600.0,NaN,YEARLY,2025-04-17 16:05:13
119230,3902300343,Bachelor’s degree in chemistry or physical sci...,Must have good oral and written communication ...,NaN,NaN,60000.0,82500.0,NaN,YEARLY,2025-04-17 16:05:13
119231,3904412353,"5+ years of Business Execution, Implementation...","Excellent verbal, written, and interpersonal c...",NaN,NaN,NaN,NaN,NaN,WEEKLY,2025-04-17 16:05:13
119232,3902940514,Bachelor’s Degree or Associate Degree or Medic...,"Strong interpersonal communication, influencin...",Bachelor’s Degree or Associate Degree or Medic...,NaN,50000.0,75000.0,NaN,NaN,2025-04-17 16:05:13


In [9]:
BATCH_SIZE = 20

while True:
    # Sample rows that are not already extracted
    df_unextracted = df_jobs_clean[~df_jobs_clean['job_id'].isin(df_extracted_job_attributes['job_id'])]
    if df_unextracted.empty:
        print("No more job descriptions to process")
        break
    print(f"No. of unprocessed job descriptions: {len(df_unextracted)}")
    df_jobs_batch = df_unextracted.sample(min(BATCH_SIZE, len(df_unextracted)), random_state=42)

    # Extract attributes
    job_ids = df_jobs_batch['job_id'].tolist()
    job_descriptions = [clean_jd(jd) for jd in df_jobs_batch['description'].tolist()]
    print(f"Extracting JD attributes for {len(df_jobs_batch)} jobs of IDs: {job_ids}...")
    try:
        parsed_job_descriptions = extract_jd_attributes(job_ids, job_descriptions)
        if parsed_job_descriptions is None:
            print("Parsed job descriptions is None")
            continue
    except Exception as e:
        print("Error extracting JD attributes", e)
        continue

    # Add parsed attributes to DataFrame
    df = df_extracted_job_attributes
    for job in parsed_job_descriptions.jobs:
        try:
            id = int(job.job_id)
            if ~(df['job_id'] == 1).any():
                df = pd.concat([df, pd.DataFrame([id], columns=['job_id'])], ignore_index=True)

            df.loc[
                df['job_id'] == id, [
                    EXTRACTED_EXPERIENCE_REQUIREMENT_COL,
                    EXTRACTED_SKILL_REQUIREMENT_COL,
                    EXTRACTED_EDUCATION_REQUIREMENT_COL,
                    EXTRACTED_CERTIFICATION_REQUIREMENT_COL,
                    EXTRACTED_MIN_SALARY_COL,
                    EXTRACTED_MAX_SALARY_COL,
                    EXTRACTED_SALARY_COL,
                    EXTRACTED_PAY_PERIOD_COL,
                    EXTRACTED_DATETIME_COL
                ]
            ] = [
                job.experience_requirements,
                job.skill_requirements,
                job.education_requirements,
                job.certification_requirements,
                parse_float(job.min_salary) if job.min_salary else None,
                parse_float(job.max_salary) if job.max_salary else None,
                parse_float(job.salary) if job.salary else None,
                job.pay_period,
                pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
            ]
        except Exception as e:
            print(f"Error processing job ID {job.job_id}", e)
            continue

        df_extracted_job_attributes = df

    # Save the updated DataFrame to a new CSV file
    print(f"Saving extracted JD attributes to extracted_job_attributes.csv for {len(df_jobs_batch)} jobs of IDs: {job_ids}...")
    save_dataframe_safely(df_extracted_job_attributes, "data/extracted_job_attributes.csv")


No more job descriptions to process


## Normalize Extracted Salaries

In [10]:
df_jobs_processed = df_jobs_clean.merge(df_extracted_job_attributes, on='job_id', how='left')
df_jobs_processed

,job_id,title,location,location_state,description,formatted_work_type,formatted_experience_level,remote_allowed,company_industries,company_country,...,pay_period,extracted_experience_requirement,extracted_skill_requirement,extracted_education_requirement,extracted_certification_requirement,extracted_min_salary,extracted_max_salary,extracted_salary,extracted_pay_period,extracted_datetime
0,921716,Marketing Coordinator,"Princeton, NJ",NJ,Job descriptionA leading real estate firm in N...,Full-time,Unknown,0.0,Real Estate,US,...,HOURLY,Marketing: 1 year (Preferred)Graphic design: 2...,"Please, be proficient in Adobe Creative Cloud ...",NaN,NaN,NaN,20.0,18.0,HOURLY,2025-04-17 16:02:34
1,1829192,Mental Health Therapist/Counselor,"Fort Collins, CO",CO,"At Aspen Therapy and Wellness , we are committ...",Full-time,Unknown,0.0,Unknown,Unknown,...,HOURLY,NaN,Conducting intake assessmentsDeveloping and im...,A graduate level psychological counseling-rela...,NaN,NaN,NaN,30.0,HOURLY,2025-04-17 16:03:04
2,10998357,Assitant Restaurant Manager,"Cincinnati, OH",OH,The National Exemplar is accepting application...,Full-time,Unknown,0.0,Restaurants,US,...,YEARLY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-17 15:51:48
3,23221523,Senior Elder Law / Trusts and Estates Associat...,"New Hyde Park, NY",NY,Senior Associate Attorney - Elder Law / Trusts...,Full-time,Unknown,0.0,Law Practice,US,...,YEARLY,10-15 years of experienceExperience with vario...,Strong analytical and problem-solving skillsAb...,Juris Doctor degree (J.D.) from an accredited ...,NaN,140000.0,175000.0,NaN,NaN,2025-04-16 23:12:49
4,35982263,Service Technician,"Burlington, IA",IA,Looking for HVAC service tech with experience ...,Full-time,Unknown,0.0,Unknown,Unknown,...,YEARLY,Minimum 5 yrs. on the job with mechanical lice...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-15 20:13:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119217,3906267117,Title IX/Investigations Attorney,"Walnut Creek, CA",CA,Our Walnut Creek office is currently seeking a...,Full-time,Mid-Senior level,0.0,Law Practice,US,...,YEARLY,five (5) to seven (7) years practicing as an a...,"Strong desire to learn, passion to work with p...",NaN,NaN,NaN,NaN,NaN,NaN,2025-04-17 00:34:26
119218,3906267126,"Staff Software Engineer, ML Serving Platform",United States,Unknown,About Pinterest:\n\nMillions of people across ...,Full-time,Mid-Senior level,1.0,Software Development,US,...,NaN,Hands-on experience building large-scale ML us...,"Fluency in Python and C++, familiarity with at...",NaN,NaN,148049.0,304496.0,NaN,YEARLY,2025-04-16 21:18:39
119219,3906267131,"Account Executive, Oregon/Washington","Spokane, WA",WA,Company Overview\n\nEPS Learning is a leading ...,Full-time,Mid-Senior level,1.0,Education Administration Programs,US,...,NaN,Possess 2 to 4 years of prior experience selli...,Exhibit strong analytical and organizational s...,"Hold a bachelor's degree in education, busines...",NaN,NaN,NaN,NaN,NaN,2025-04-16 21:26:39
119220,3906267195,Business Development Manager,"Texas, United States",Unknown,The Business Development Manager is a 'hunter'...,Full-time,Unknown,1.0,Industrial Machinery Manufacturing,GB,...,NaN,Established relationships working with oil & g...,Decision MakingCommercial AwarenessCustomer Kn...,Some college or technical background would be ...,NaN,NaN,NaN,NaN,NaN,2025-04-13 16:47:31


In [11]:
def get_extracted_normalized_salary(row: pd.Series, min_salary = 1e4, max_salary = 1e6, max_salary_range_spread = 4) -> float:
    pay_period = row['extracted_pay_period']
    salary: float = np.nan

    if row['extracted_min_salary'] and row['extracted_max_salary']:
        (extracted_min_salary, extracted_max_salary) = (row['extracted_min_salary'], row['extracted_max_salary'])
        if extracted_min_salary > extracted_max_salary:
            return np.nan

        if (extracted_max_salary - extracted_min_salary) / extracted_min_salary > max_salary_range_spread:        
            return np.nan
        
        salary = (extracted_min_salary + extracted_max_salary) / 2
    elif row['extracted_salary']:
        salary = row['extracted_salary']
    
    if np.isnan(salary) or not pay_period: return np.nan
    
    if pay_period == 'HOURLY':
        salary = salary * 40 * 52
    elif pay_period == 'WEEKLY':
        salary = salary * 52
    elif pay_period == 'BIWEEKLY':
        salary = salary * 26
    elif pay_period == 'MONTHLY':
        salary = salary * 12
    elif pay_period == 'YEARLY':
        pass
    else:
        return np.nan

    if salary < min_salary or salary > max_salary:
        return np.nan

    return salary

In [12]:
df_jobs_processed['extracted_normalized_salary'] = df_jobs_processed.apply(get_extracted_normalized_salary, axis=1)

## Export Processed Jobs Data

In [13]:
# Sample a row to view extracted attributes
with pd.option_context('display.max_colwidth', None):
    display(df_jobs_processed[df_jobs_processed['extracted_normalized_salary'].notna()].sample(1, random_state=42).T)

,55774
job_id,3901943969
title,Custodian
location,"Troy, OH"
location_state,OH
description,"Description\n\nPosition at SBM Management\n\nSBM Management is looking for a Custodian to help them shine! We are searching for a hardworking dependable individual to join the team as our company continues to grow! We have an immediate opening for a custodian that wants to deliver exceptional customer satisfaction. We are looking for custodial members that have strong customer service skills and can work successfully with other team members.\n\nOur innovative employee programs, supportive management structure, and extensive career advancement opportunities make SBM a great place to work. As a result, our turnover rate is one-fourth the national average. We respect and promote the professional and personal growth of our employees and are committed to the success of one another. Teamwork, integrity and compassion are core values of our company, and we go to great lengths to ensure that our employees are satisfied and rewarded for the work that they do.\n\nResponsibilities\n\nFloor care (vacuuming, mopping, carpet spotting)Surface care (dusting, sanitizing, glass/mirror/window cleaning, metal/wood polishing, clean blinds and draperies)Waste removal (empty trash and recycle bins, transport trash and waste to proper disposal areas)Light maintenance (replace light bulbs, restock supply cabinet/room)\n\nQualifications\n\nCustodial experience is preferred but not requiredPassing a drug test will be required\n\nCompensation: $13.50 - $14.50 per hour\n\nShift: Monday - Friday, 7:00AM - 3:30PM, Monday - Friday 4:00PM - 12:30AM, Monday - Friday 4:00PM - 8:00PM\n\nSBM Management Services, LP and its affiliates are proud to be equal opportunity workplaces. We are committed to equal employment opportunity regardless of race, sex, color, ancestry, religion, national origin, sexual orientation, citizenship, age, marital status, disability, gender identity, Veteran status, or other legally protected status."
formatted_work_type,Full-time
formatted_experience_level,Entry level
remote_allowed,0.0
company_industries,Facilities Services
company_country,US


In [14]:
df_jobs_processed.to_csv("data/jobs_processed.csv", index=False)

## Export Company Industries

In [19]:
df_jobs_processed['company_industries'].value_counts().to_csv("data/company_industries.csv", index=True)